# Computergestützte Mathematik zur Analysis

28.01.2021

Prof. Dr. Rüdiger Braun

In [ ]:
from sympy import *
import numpy as np
from matplotlib import pyplot as plt
init_printing()

# Differentialgleichungen höherer Ordnung

In [ ]:
x = Symbol('x')
y = Function('y')

In [ ]:
dgl = Eq(y(x).diff(x,2), -y(x).diff(x) - y(x))
dgl

In [ ]:
lsg = dsolve(dgl)
lsg

In [ ]:
phi = lsg.rhs

Anfangsbedingungen $y(1)=2$, $y'(1)=3$

Kann `dsolve` jetzt doch

In [ ]:
lsg = dsolve(dgl, ics={y(1): 2, y(x).diff(x).subs(x, 1): 3})
lsg

In [ ]:
phi = lsg.rhs

In [ ]:
plot(phi);

# Besselfunktionen

In [ ]:
x = S('x')
y = Function('y')
nu = Symbol('nu', positive=True)  #  nicht nötig

In [ ]:
dgl = Eq(y(x).diff(x,2) + y(x).diff(x)/x + (1 - nu**2/x**2)*y(x), 0)
dgl

In [ ]:
lsg = dsolve(dgl)
lsg

In [ ]:
phi = lsg.rhs
print(phi)

$J_\nu$: Besselfunktion erster Art der Ordnung $\nu$

$Y_\nu$: Besselfunktion zweiter Art der Ordnung $\nu$, Webersche Funktion

In [ ]:
nu_n = 2
plot(besselj(nu_n, x), bessely(nu_n, x), (x, 0, 60), ylim=(-4,1));

Wronskische

In [ ]:
M = Matrix(2,2,[besselj(nu, x), bessely(nu,x), besselj(nu,x).diff(x), bessely(nu,x).diff(x)])
M

In [ ]:
M.det()

In [ ]:
besselsimp(M.det())

In [ ]:
plot(M.det().subs(nu,Rational(3,2))*x, (x,0,5));

In [ ]:
classify_ode(dgl)

In [ ]:
lsg_ser = dsolve(dgl.subs(nu,Rational(3,2)), func=y(x), hint='2nd_power_series_regular')
lsg_ser

In [ ]:
yy = lsg_ser.rhs
ps = yy.subs(S('C1'),0).subs(S('C2'), 1)
qs = yy.subs(S('C1'),1).subs(S('C2'), 0)
ps, qs

In [ ]:
M = Matrix(2,2,[ps, qs, ps.diff(x), qs.diff(x)])
M.det()

# Gekoppelte Pendel

In [ ]:
y = Function('y')
w = Function('w')
t = Symbol('t', positive=True)

In [ ]:
a = Rational(1,7)
b = a

In [ ]:
dgl1 = Eq(y(t).diff(t,2), a*w(t) - (1+a)*y(t))
dgl2 = Eq(w(t).diff(t,2), b*y(t) - (1+b)*w(t))
dgs = {dgl1, dgl2}
dgs

In [ ]:
lsg = dsolve(dgs)
lsg

In [ ]:
phi_c = lsg[0].rhs
psi_c = lsg[1].rhs

Bestimme reelles Fundamentalsystem

In [ ]:
Cs = []
for j in range(4):
    Cs.append(S(f'C{j+1}'))
Cs

In [ ]:
ersetzungen = [
    {Cs[0]: 1, Cs[1]: 1, Cs[2]: 0, Cs[3]: 0},
    {Cs[0]: I, Cs[1]: -I, Cs[2]: 0, Cs[3]: 0},
    {Cs[0]: 0, Cs[1]: 0, Cs[2]: 1, Cs[3]: 1},
    {Cs[0]: 0, Cs[1]: 0, Cs[2]: I, Cs[3]: -I},
]

In [ ]:
fus = []
for s in ersetzungen:
    fus.append([phi_c.subs(s).expand(complex=True), 
                psi_c.subs(s).expand(complex=True)])

In [ ]:
fus

In [ ]:
f = fus[0][0] + fus[2][0]
g = fus[0][1] + fus[2][1]
f, g

In [ ]:
print(f)

In [ ]:
plot(f, g+1, (t,-100, 100));

# Matrixexponentiale

Wir fassen die Variablen $y$ und $w$ zu $Y$ zusammen als

$$
Y = \begin{pmatrix}
y \\ y' \\ w \\ w'
\end{pmatrix}
$$

Dann
$$
Y' = \begin{pmatrix}
    0 & 1 & 0 & 0 \\
    -1-a & 0 & a & 0 \\
    0 & 0 & 0 & 1 \\
    b & 0 & -1-b & 0
\end{pmatrix}
$$

In [ ]:
a,b = symbols('a,b')
M_ab = Matrix(4,4, [0,1,0,0,-1-a,0,a,0,0,0,0,1,b,0,-1-b,0])
M_ab

In [ ]:
M = M_ab.subs(a, Rational(1,7)).subs(b, Rational(1,7))
M_exp = (M*t).exp()
M_exp

In [ ]:
M_exp.det()

In [ ]:
M_exp.det().simplify()

In [ ]:
fus = M_exp[0,:]
fus

In [ ]:
M.eigenvals()

In [ ]:
T, J = M.jordan_form()
T, J

In [ ]:
d = []
for j in range(4):
    d.append(exp(t*J[j,j]))
d

In [ ]:
diag(J)

In [ ]:
M2 = T * Matrix.diag(d) * T**(-1)

In [ ]:
expand(M2, complex=True)